# Heat map visualization

### Imports

In [1]:
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import cartopy as ctpy
import matplotlib.pyplot as plt

from read_data import *
from create_product import *

### Constants

In [2]:
OUTPUT_PATH = "out"

### Loading COVID Data

In [3]:
data, timesteps = get_covid_confirmed_global()

print(data)

    Province/State      Country/Region        Lat        Long  1/22/20  \
0              NaN         Afghanistan  33.939110   67.709953        0   
1              NaN             Albania  41.153300   20.168300        0   
2              NaN             Algeria  28.033900    1.659600        0   
3              NaN             Andorra  42.506300    1.521800        0   
4              NaN              Angola -11.202700   17.873900        0   
..             ...                 ...        ...         ...      ...   
269            NaN             Vietnam  14.058324  108.277199        0   
270            NaN  West Bank and Gaza  31.952200   35.233200        0   
271            NaN               Yemen  15.552727   48.516388        0   
272            NaN              Zambia -13.133897   27.849332        0   
273            NaN            Zimbabwe -19.015438   29.154857        0   

     1/23/20  1/24/20  1/25/20  1/26/20  1/27/20  ...  2/28/21  3/1/21  \
0          0        0        0       

### Confirmed cases heat map

In [4]:
# https://scitools.org.uk/cartopy/docs/v0.13/matplotlib/advanced_plotting.html
# https://blog.mapbox.com/visualizing-the-progression-of-the-2019-ncov-outbreak-66763eb59e79

def create_world_mesh(lats, lons, vals, res=1.0):
    """
    Creates a 2D mesh of the data contained in vals at the corresponding points in lats and lons
    Optionally, 
    
    Assumes lats, lons and vals are 1D arrays of the same size
    
    Res must be a multiple of 10. Other values are not guaranted to work! 
    x.0      - Lower resolution
    1.0      - Country or large region
    0.1      - Large city or district
    0.01     - Town or village
    0.001    - Neighborhood, street
    0.0001   - Individual street, large buildings
    0.00001  - Individual trees, houses
    0.000001 - Individual humans
    """
    
    mesh_w = int(360/res)
    mesh_h = int(180/res)
    
    lats_mesh = np.linspace(-90,90, mesh_h)
    lons_mesh = np.linspace(-180,180, mesh_w)
    
    mesh = np.zeros((mesh_h, mesh_w))
        
    # Set all mesh values
    for i in range(vals.shape[0]):
        lat = lats.iloc[i]
        lon = lons.iloc[i]
        val = vals.iloc[i]
        
        mesh_lat = int(((lat+90.0)/180)*mesh_h)
        mesh_lon = int(((lon+180.0)/360)*mesh_w)
        mesh[mesh_lat, mesh_lon] += val
    
    return lats_mesh, lons_mesh, mesh


def heat_map(data, timesteps, step, res=1.0):
    """
    Creates a heatmap figure for the given timestep "step"
    """
    
    lats = data['Lat']
    lons = data['Long']
    time = timesteps[step]
    vals = data[time]
    lats_mesh, lons_mesh, val_mesh = create_world_mesh(lats, lons, vals, res=res)

    # Create the figure
    fig = plt.figure(figsize=(16,8), dpi=90)
    ax = plt.axes(projection=ccrs.PlateCarree())
    ax.set_extent([-180,180,-90,90])
    ax.coastlines()
    
    # This masks the ocean
    ax.add_feature(ctpy.feature.OCEAN, zorder=99, edgecolor='k', color="white")
    ax.add_feature(ctpy.feature.BORDERS, zorder=99, edgecolor='black')
    ax.add_feature(ctpy.feature.COASTLINE, zorder=99, edgecolor='black')
    ax.set_global()
    
    plt.contourf(lons_mesh, lats_mesh, val_mesh, 128, cmap='summer', transform=ccrs.PlateCarree())
    plt.text(-170,78, time, backgroundcolor="black", color="white", fontsize="xx-large", zorder=100, transform=ccrs.PlateCarree())

    return fig

### Generate sample for testing

In [6]:
test_imgs = []
test_timesteps = timesteps[-10:]
for t in range(len(test_timesteps)):
    fig = heat_map(data, test_timesteps, t, res=1.0)
    fig.savefig(f"{OUTPUT_PATH}/heat_map_test_{t}.png", format='png')
    img = plt_to_img(fig)
    test_imgs.append(img)

    # Preserve memory by removing figures. No need to retain them
    plt.clf()
    plt.close(fig)
    
imgs_to_video(test_imgs, f"{OUTPUT_PATH}/heat_map_test_video.avi", framerate=1)

### Generate images

In [8]:
imgs = []
for t in range(len(timesteps)):
    fig = heat_map(active_global_shape, active_global_timesteps, t, max_val)
    fig.savefig(f"{OUTPUT_PATH}/heat_map_{t}.png", format='png')
    img = plt_to_img(fig)
    imgs.append(img)

    # Preserve memory by removing figures. No need to retain them
    plt.clf()
    plt.close(fig)

### Generate video

In [9]:
imgs_to_video(imgs, f"{OUTPUT_PATH}/heat_map_video.avi", framerate=15)